# Univariate time series classification with sktime-dl
[Github](https://github.com/sktime/sktime-dl)

In this notebook, we use sktime-dl to perform for univariate time series classification by deep learning.

In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sktime.datasets import load_gunpoint, load_italy_power_demand
from sklearn.model_selection import GridSearchCV
from sktime.pipeline import Pipeline
from sktime.transformers.compose import Tabulariser, RowwiseTransformer

from sktime_dl.deeplearning import CNNClassifier

sns.set_style('whitegrid') 

# Load a dataset

In [ ]:
X_train, y_train = load_italy_power_demand(split='TRAIN', return_X_y=True)
X_test, y_test = load_italy_power_demand(split='TEST', return_X_y=True)
X_train.head()

In [ ]:
def plot_data_samples(X, y, sample_numbers):
    ''' 
    Plot the time series data relating to the input list of sample numbers.

    sample_numbers: list of integers
        E.g. [1, 7, 22, 42]
    '''
    
    Xt = Tabulariser().transform(X)
    unique_labels = np.unique(y).astype(int)
    num_classes = len(unique_labels)
    if num_classes<=4:
        class_colors = ['red', 'blue', 'green' , 'orange']
    else:
        class_colors = sns.color_palette(n_colors=num_classes)

    fig, ax = plt.subplots()
    for i in sample_numbers:
        print('sample', i, 'class', str(y[i]))
        color_num = y[i].astype(int) - unique_labels.min()
        Xt.iloc[i].plot(label=str(y[i]), color=class_colors[color_num])

    print('')
    plt.ylim([-3.5, 3.5])
    if num_classes<=2:
        title = class_colors[0]+' : class '+str(unique_labels[0])
        title = title + '\n'+class_colors[1]+' : class '+str(unique_labels[1])
        plt.title(title)
    ax.set_ylabel('Data value')
    ax.set_xlabel('Data point number')

## Plot some data samples

In [ ]:
plot_data_samples(X_train, y_train, [0, 1, 2, 3])

# Train a single deep neural network classifier
Here we choose to use the CNN (convolutional neural network) classifier. Other classifiers provided by sktime-dl include MLP, ResNet and InceptionTime.

In [ ]:
network = CNNClassifier(nb_epochs=200, verbose=False)
network.fit(X_train, y_train)
network.score(X_test, y_test)

## Save the model to file

In [ ]:
network.model.save("temp_model.h5")

# Grid Search
sktime-dl is compatible with scikit-learn and can use sklearn's GridSearchCV.

Here we search over two parameters, number of epochs and CNN kernel size.

In [ ]:
param_grid = {'nb_epochs': [50, 100],
              'kernel_size': [5, 7, 9] }
grid = GridSearchCV(network, param_grid=param_grid, cv=5)   
grid.fit(X_train, y_train)
    
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))